In [7]:
import sqlite3
import json
import datetime
import pandas as pd
import numpy as np
from trading_ig.config import config
import yfinance as yf
from pathlib import Path
from trading_ig import IGService, IGStreamService
from trading_ig.lightstreamer import Subscription
import time
from autoIG.epics import US_CRUDE_OIL_EPIC,BITCOIN_EPIC,GOLD_EPIC

from autoIG.utils import load_model,read_stream_,parse_item
model = load_model("base_model/base_model.pkl")

# Set up Subscription
ig_service = IGService(
    config.username,
    config.password,
    config.api_key,
    config.acc_type,
    acc_number=config.acc_number,
)
ig_stream_service = IGStreamService(ig_service)
ig_stream_service.create_session()
sub = Subscription(
    mode="MERGE", items=["L1:" + US_CRUDE_OIL_EPIC], fields=["UPDATE_TIME", "BID", "OFFER"]
)
from autoIG.utils import parse_item


In [8]:
config = dict()
config["r_threshold"] = 1.00113


def on_update(item):
    parse_item(item).to_csv("stream_.csv", mode="a", header=False)
    stream_ = read_stream_(nrows=-2)
    predictions = model.predict(stream_[["BID"]])
    latest_prediction = predictions[-1]
    print(f"Latest prediction: {latest_prediction}")
    if latest_prediction > config["r_threshold"]:
        print("We BUY!")
    stream_.to_csv("stream_.csv", mode="w", header=False)
    return None

sub.addlistener(on_update)
ig_stream_service.ls_client.subscribe(sub)
time.sleep(3)
ig_stream_service.disconnect()


Latest prediction: 1.0010820889602312
Latest prediction: 1.001104818567109
Latest prediction: 1.0010979996850458
Latest prediction: 1.00111391040986
Latest prediction: 1.0010979996850458
Latest prediction: 1.0010911808029823
Latest prediction: 1.001104818567109
Latest prediction: 1.0010979996850458
Latest prediction: 1.001104818567109
DISCONNECTED FROM LIGHTSTREAMER


In [2]:
ig_service